In [4]:
pip install opencv-python

In [5]:
pip install keras

In [6]:
pip install scikit-learn

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [2]:
def load_videos(folder):
    videos = []
    for filename in os.listdir(folder):
        if filename.endswith(".mp4"):
            videos.append(os.path.join(folder, filename))
    return videos

real_videos = load_videos("Real")
deepfake_videos = load_videos("Deepfake")


In [3]:
def read_video(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    return frames

real_frames = [read_video(video) for video in real_videos]
deepfake_frames = [read_video(video) for video in deepfake_videos]


In [4]:
X = real_frames + deepfake_frames
y = [0] * len(real_frames) + [1] * len(deepfake_frames)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
def extract_features(frames):
    features = []
    for frame in frames:
        hist_r = cv2.calcHist([frame], [0], None, [256], [0, 256]).flatten()
        hist_g = cv2.calcHist([frame], [1], None, [256], [0, 256]).flatten()
        hist_b = cv2.calcHist([frame], [2], None, [256], [0, 256]).flatten()
        features.append(np.concatenate([hist_r, hist_g, hist_b]))
    return features

X_train = [extract_features(frames) for frames in X_train]
X_test = [extract_features(frames) for frames in X_test]


In [7]:
clf = SVC()
X_train_flattened = []
y_train_flattened = []
for X_frames, y_label in zip(X_train, y_train):
    X_train_flattened.extend(X_frames)
    y_train_flattened.extend([y_label] * len(X_frames))

clf.fit(X_train_flattened, y_train_flattened)


SVC()

In [8]:
test_video_frames = read_video("test_video.mp4")
test_video_features = extract_features(test_video_frames)
predictions = clf.predict(test_video_features)


In [9]:
for prediction in predictions:
    if prediction == 0:
        print("Real")
    else:
        print("Deepfake")


Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real


In [10]:
# Параметры модели
parameters = [
    {"kernel": "linear", "C": 1.0},
    {"kernel": "rbf", "C": 1.0, "gamma": "scale"},
    {"kernel": "poly", "C": 1.0, "degree": 3}
]

# Создание таблицы результатов
results = []

# Обучение и оценка модели для каждого параметра
for params in parameters:
    clf = SVC(**params)

    X_train_flattened = []
    y_train_flattened = []
    for X_frames, y_label in zip(X_train, y_train):
        X_train_flattened.extend(X_frames)
        y_train_flattened.extend([y_label] * len(X_frames))

    clf.fit(X_train_flattened, y_train_flattened)

    X_test_flattened = []
    y_test_flattened = []
    for X_frames, y_label in zip(X_test, y_test):
        X_test_flattened.extend(X_frames)
        y_test_flattened.extend([y_label] * len(X_frames))

    y_pred = clf.predict(X_test_flattened)
    accuracy = accuracy_score(y_test_flattened, y_pred)

    results.append({
        "Params": params,
        "Accuracy": accuracy
    })

# Вывод таблицы результатов
for result in results:
    print("Params:", result["Params"])
    print("Accuracy:", result["Accuracy"])
    print()


Params: {'kernel': 'linear', 'C': 1.0}
Accuracy: 1.0

Params: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}
Accuracy: 1.0

Params: {'kernel': 'poly', 'C': 1.0, 'degree': 3}
Accuracy: 0.8640054127198917

